# Toxic Comment Classification
### Introduction
This project focuses on building a model to classify toxic comments on social media platforms. The dataset, sourced from Kaggle, includes various labels indicating different types of toxicity such as toxic, severe toxic, obscene, threat, insult, and identity hate. The objective is to predict the probability of each type of toxicity for a given comment.


## 1. Importing Modules and Loading the Data

First, we import the necessary libraries:
- **os**: For interacting with the operating system, such as file operations.
- **tensorflow**: To build and train the deep learning model. TensorFlow is a powerful library for numerical computation and is especially well-suited for large-scale machine learning.
- **pandas**: For data manipulation and analysis. Pandas provides data structures like DataFrames, which are essential for handling and processing data efficiently.
- **numpy**: For numerical operations. NumPy supports large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays.

Next, we check if TensorFlow can access the GPUs, which can significantly speed up the training process. The number of GPUs available is printed to the console.


In [1]:
import os
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 1


In [2]:
import pandas as pd
import numpy as np

np.random.seed(0)

We load the dataset using Pandas. The methods `head()`, `info()`, `describe()`, and `isnull().sum()` provide an initial exploration of the dataset:
- **head()**: Displays the first few rows of the dataset, giving a glimpse of the data structure and values.
- **info()**: Provides a concise summary of the DataFrame, including the data types and non-null values.
- **describe()**: Generates descriptive statistics such as mean, standard deviation, and percentiles for numerical columns.
- **isnull().sum()**: Checks for missing values in each column, which is crucial for data cleaning.

In [3]:
df = pd.read_csv('dataset/train.csv')

In [4]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [6]:
df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
df.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

## 2. Data Preprocessing

### Vectorization of Text Data

Text data needs to be converted into a numerical format for the model to process it. We use TensorFlow's `TextVectorization` layer to transform text into integer sequences. This layer is adaptable, allowing it to build a vocabulary based on the dataset.

In [8]:
from tensorflow.keras.layers import TextVectorization

In [9]:
x = df['comment_text']
# .values to convert to numpy array
y = df.drop(['id', 'comment_text'], axis=1).values


- **MAX_FEATURES**: Sets the maximum size of the vocabulary. This limits the number of unique words considered, reducing the complexity and size of the model.
- **output_sequence_length**: Specifies the length of the output sequence. Shorter texts are padded, and longer texts are truncated to ensure uniform input size.

In [10]:
# number of words in vocabulary
MAX_FEATURES = 200000

In [11]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=2000,
                               output_mode='int')

In [12]:
vectorizer.adapt(x.values)

In [13]:
# vectorizer.get_vocabulary()[:5]

In [14]:
vectorizer('The first rule of Fight Club is')[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([   2,  113,  623,    4, 1275], dtype=int64)>

In [15]:
vectorized_text = vectorizer(x.values)

In [16]:
vectorized_text.shape

TensorShape([159571, 2000])

### Creating TensorFlow Dataset

We create a TensorFlow dataset from the vectorized text and labels. This dataset is prepared for training by shuffling, batching, and prefetching:
- **cache()**: Caches the dataset in memory for faster access during training.
- **shuffle()**: Randomly shuffles the dataset to ensure that the training data is not in any specific order, which helps the model generalize better.
- **batch()**: Groups the data into batches. Batch processing is efficient and helps in faster computation.
- **prefetch()**: Allows the model to fetch batches in the background while the current batch is being processed, improving performance.


In [17]:
# map -> cache -> shuffle -> batch -> prefetch
# instantiate dataset = from_tensor_slices / list_files
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y)) \
        .cache() \
        .shuffle(160000) \
        .batch(16) \
        .prefetch(8)
# 160000 = buffersize, prefetch helps prevent bottlenecks


In [18]:
x_batch, y_batch = dataset.as_numpy_iterator().next()
print(x_batch.shape, y_batch.shape)

(16, 2000) (16, 6)


In [19]:
n = len(dataset)
n

9974

We then split the dataset into training, validation, and test sets. This ensures that we have separate data for training the model and evaluating its performance.

In [20]:
train = dataset.take(int(n * 0.7))
val = dataset.skip(int(n * 0.7)).take(int(n * 0.2))
test = dataset.skip(int(n * 0.9)).take(int(n * 0.1))

In [21]:
print(len(train), len(val), len(test))

6981 1994 997


## 3. Building the Sequential Model

We build a Sequential model using Keras. The Sequential API allows us to stack layers sequentially.

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Bidirectional, Dense


### Embedding Layer

The embedding layer converts integer-encoded words into dense vectors of fixed size. This is essential for capturing the semantic meaning of words and their relationships.

### Bidirectional LSTM Layer

LSTMs (Long Short-Term Memory networks) are effective for sequential data, such as text. A bidirectional LSTM processes the sequence in both forward and backward directions, capturing context from both ends.

### Dense Layers

Dense (fully connected) layers are used to extract features from the LSTM output. We use multiple dense layers with ReLU activation to introduce non-linearity, allowing the model to learn complex patterns.

### Output Layer

The final dense layer with a sigmoid activation function predicts the probabilities of the six types of toxicity. Sigmoid is used because we are dealing with multi-label classification, where each label is independent.


In [23]:
model = Sequential()

# create embedding layer
model.add(Embedding(MAX_FEATURES+1, 32))

# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))

# Feature extractor fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

# final layer, 6 = 6 labels
model.add(Dense(6, activation='sigmoid'))

<img src="model-architecture.png" alt="Model Architecture" title="Title" width="30%" height="30%">


### Compiling the Model

We compile the model with the Binary Crossentropy loss function, which is suitable for multi-label classification. The Adam optimizer is used for its efficiency and adaptive learning rate properties.

In [24]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        


### Training the Model

We train the model on the training dataset, with validation on the validation set. Training involves multiple epochs, where the model iteratively learns from the data. The model is saved after training for later use.

In [26]:
# hist = model.fit(train, epochs=2, validation_data=val)

In [27]:
# model.save('toxic-comment-classifier-new.h5')

In [28]:
from tensorflow.keras.models import load_model

model = load_model('toxic-comment-classifier.h5')


In [29]:
from matplotlib import pyplot as plt

In [30]:
# plt.figure(figsize=(10, 6))
# pd.DataFrame(hist.history()).plot()
# plt.show()

## 4. Making Predictions

To make predictions, we preprocess the input text using the vectorizer and predict the probabilities of each type of toxicity using the trained model. This allows us to classify new comments based on the learned patterns.

In [38]:
input_text = 'You fucking moron! I will kill you.'
input_text

'You fucking moron! I will kill you.'

In [39]:
def convert_text(text):
    return np.expand_dims(vectorizer(text), 0)

In [40]:
res = model.predict(convert_text(input_text))
res

1/1 [==============================] - 9s 9s/step


array([[0.9984818 , 0.32446837, 0.94736516, 0.08575991, 0.9035318 ,
        0.25237027]], dtype=float32)

In [46]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [41]:
batch_x, batch_y = test.as_numpy_iterator().next()

In [48]:
res = (model.predict(batch_x) > 0.5).astype('int')
res

1/1 [==============================] - 0s 255ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

## 5. Evaluating the Model

We evaluate the model using metrics such as Precision, Recall, and Categorical Accuracy. These metrics help us understand the performance of the model in terms of true positives, false positives, and overall accuracy.

In [49]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [50]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()


- **Precision**: Measures the accuracy of positive predictions. It is the ratio of true positive predictions to the total positive predictions.
- **Recall**: Measures the ability to identify all positive instances. It is the ratio of true positive predictions to the total actual positives.
- **Categorical Accuracy**: Measures the overall accuracy of the model in predicting the correct category.

By iterating over the test dataset, we update the metrics and print the final results.

In [52]:
for batch in test.as_numpy_iterator():
    # actual value
    actual_x, actual_y = batch

    # predicted value
    y_pred = model.predict(actual_x)

    #flatten the prediction
    actual_y = actual_y.flatten()
    y_pred = y_pred.flatten()

    pre.update_state(actual_y, y_pred)
    re.update_state(actual_y, y_pred)
    acc.update_state(actual_y, y_pred)


1/1 [==============================] - 0s 93ms/step


In [57]:
def conv_res(res):
    return res.result().numpy()
print(f'         Precision = {conv_res(pre)} \n \
        Recall = {conv_res(re)} \n \
        Accuracy = {conv_res(acc)}')

         Precision = 0.8309716582298279 
         Recall = 0.7321640849113464 
         Accuracy = 0.5025075078010559



## 6. Integrating with Gradio

Gradio provides a simple way to create interactive interfaces for machine learning models. By setting up a Gradio interface, users can input comments and get the predicted probabilities of each type of toxicity.

- **predict_toxicity**: A function that takes a comment, preprocesses it, and returns the predicted probabilities.
- **gr.Interface**: Creates the interface with specified input and output types. This interface can be launched as a web application for user interaction.


In [66]:
import gradio as gr

In [107]:
def predict_toxicity(comment):
    vectorized_comment = vectorizer([comment])
    result = model.predict(vectorized_comment)

    score = ''
    for idx, col in enumerate(df.columns[2:]):
        score += f'{col}: {result[0][idx] > 0.5}<br>'
    return score

In [108]:
interface = gr.Interface(
    fn=predict_toxicity,
    inputs=gr.Textbox(lines=2, placeholder='Predict Toxicity of comment'),
    outputs=gr.HTML(),
    theme=gr.themes.Base())


In [109]:
interface.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 218ms/step
